# Tools and OpenAI Functions
We'll focus on SerpAPI using OpenAI as our Agent.

An agent decides *when* to use tools based on the needs of the time.

In [1]:
%pip install -Uq langchain google-search-results numexpr langsmith langchainhub

  Using cached google_search_results-2.4.2-py3-none-any.whl
  Using cached numexpr-2.8.8-cp310-cp310-win_amd64.whl.metadata (8.1 kB)
  Using cached langchainhub-0.1.14-py3-none-any.whl.metadata (478 bytes)
  Using cached types_requests-2.31.0.10-py3-none-any.whl.metadata (1.8 kB)
   ---------------------------------------- 0.0/794.3 kB ? eta -:--:--
   -------- ------------------------------- 174.1/794.3 kB 3.5 MB/s eta 0:00:01
   ---------------------------------- ----- 686.1/794.3 kB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 794.3/794.3 kB 7.2 MB/s eta 0:00:00
Using cached numexpr-2.8.8-cp310-cp310-win_amd64.whl (96 kB)
   ---------------------------------------- 0.0/46.3 kB ? eta -:--:--
   ---------------------------------------- 46.3/46.3 kB 2.3 MB/s eta 0:00:00
Using cached langchainhub-0.1.14-py3-none-any.whl (3.4 kB)
Using cached types_requests-2.31.0.10-py3-none-any.whl (14 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 

In [2]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "serpsearchingtooltesting"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""
# We will use OpenAI as the Agent
os.environ["OPENAI_API_KEY"] = ""

In [3]:
# This step is for an outdated version of LangChain/LangSmith, \
    # but it's still worth leaving in for your knowledge and reference 
from langsmith import Client

# Initialize the LangSmith client
client = Client()

In [4]:
from operator import itemgetter

from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function

# Initialize the LLM using a Chat model
llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0.2)

# Load your tools by using the built-in ones for quick testing. You may wish to try building a custom tool
# llm-math is an especially powerful built-in tool, since LLMs can't perform math, \
    # they just predict the next token in the conversation
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Bind the tools to the LLM
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

# https://smith.langchain.com/hub/daethyra/rag-prompt
prompt = hub.pull("wfh/agent-lcel-prompt")

In [5]:
# The scratchpad is imperative for agents so they don't lose their place as they take action
def format_scratchpad(x: dict) -> str:
    return format_to_openai_functions(x["intermediate_steps"])

def get_chat_history(x: dict) -> list:
    return x.get("chat_history", [])

In [6]:
# Construct the Agent by using LangChain Expression Language to chain functionality together
agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": format_scratchpad,
        "chat_history": get_chat_history,
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
).with_config(run_name="Agent")

# Define the Agent Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
)

In [7]:
inputs = [
    "What's the distance from Earth to the moon in miles multiplied by 77?",
    "How much further away is Mars from Earth than Earth to the moon?",
    "How many stars are there in the Milky Way multiplied by the number of planets in our solar system",
    "How old is the Sun in years multiplied by the number of planets in our solar system",
    "How many more stars are there in our galaxy than in our solar system?",
    "What is the number of stars in the Andromeda Galaxy divided by the number of stars in the Milky Way?",
    "How much larger is the biggest known star compared to the Sun?",
    "What is the surface area of Earth in feet divided by the number of people on Earth?",
    "How big is a black hole in miles divided by the diameter of Earth in miles?",
    "How much larger is the biggest planet compared to the smallest planet?"
]

# Batch process the inputs using the Agent Executor
results = agent_executor.batch([{"input": x} for x in inputs], return_exceptions=True)
results

[{'input': "What's the distance from Earth to the moon in miles multiplied by 77?",
  'output': 'The distance from Earth to the moon is approximately 238,855 miles. When multiplied by 77, the result is 18,391,835 miles.'},
 ValueError('LLMMathChain._evaluate("\ndistance from Earth to Mars in miles - distance from Earth to the moon in miles\n") raised error: invalid syntax (<expr>, line 1). Please try again with a valid numerical expression'),
 {'input': 'How many stars are there in the Milky Way multiplied by the number of planets in our solar system',
  'output': 'The number of stars in the Milky Way multiplied by the number of planets in our solar system is 1,600,000,000,000.'},
 {'input': 'How old is the Sun in years multiplied by the number of planets in our solar system',
  'output': 'The age of the Sun in years multiplied by the number of planets in our solar system is 36,800,000,000 years.'},
 {'input': 'How many more stars are there in our galaxy than in our solar system?',
  '